In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pickle
import Orange

In [2]:
base_censo = Orange.data.Table('data/census_regras.csv')

In [4]:
base_censo.domain

[age, workclass, final-weight, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loos, hour-per-week, native-country | income]

In [5]:
base_divida = Orange.evaluation.testing.sample(base_censo, n=0.25)

In [6]:
base_divida

([[23, Private, 216639, Bachelors, 13, ... | <=50K],
  [38, Private, 255941, Masters, 14, ... | >50K],
  [25, Private, 181666, Assoc-acdm, 12, ... | <=50K],
  [45, Private, 126889, Bachelors, 13, ... | >50K],
  [43, Local-gov, 169203, Assoc-voc, 11, ... | <=50K],
  ...
 ],
 [[23, State-gov, 298871, Some-college, 10, ... | <=50K],
  [22, Private, 29444, 12th, 8, ... | <=50K],
  [52, Private, 122109, HS-grad, 9, ... | <=50K],
  [37, Private, 70562, 1st-4th, 2, ... | <=50K],
  [24, Private, 464103, HS-grad, 9, ... | <=50K],
  ...
 ])

In [7]:
base_divida[0]

[[23, Private, 216639, Bachelors, 13, ... | <=50K],
 [38, Private, 255941, Masters, 14, ... | >50K],
 [25, Private, 181666, Assoc-acdm, 12, ... | <=50K],
 [45, Private, 126889, Bachelors, 13, ... | >50K],
 [43, Local-gov, 169203, Assoc-voc, 11, ... | <=50K],
 ...
]

In [8]:
base_divida[1]

[[23, State-gov, 298871, Some-college, 10, ... | <=50K],
 [22, Private, 29444, 12th, 8, ... | <=50K],
 [52, Private, 122109, HS-grad, 9, ... | <=50K],
 [37, Private, 70562, 1st-4th, 2, ... | <=50K],
 [24, Private, 464103, HS-grad, 9, ... | <=50K],
 ...
]

In [9]:
base_treinamento = base_divida[1]
base_teste = base_divida[0]

In [10]:
len(base_treinamento), len(base_teste)

(24421, 8140)

In [11]:
cn2 = Orange.classification.rules.CN2Learner()
regras_censo = cn2(base_treinamento)

In [13]:
for regras in regras_censo.rule_list:
    print(regras)

IF workclass==Never-worked THEN income=<=50K 
IF workclass==Without-pay THEN income=<=50K 
IF education==Preschool THEN income=<=50K 
IF occupation==Armed-Forces THEN income=<=50K 
IF native-country==Holand-Netherlands THEN income=<=50K 
IF native-country==Outlying-US(Guam-USVI-etc) THEN income=<=50K 
IF native-country==Trinadad&Tobago THEN income=<=50K 
IF age<=22.0 AND workclass==Federal-gov THEN income=<=50K 
IF age<=22.0 AND workclass==Self-emp-inc THEN income=<=50K 
IF age<=22.0 AND education==1st-4th THEN income=<=50K 
IF age<=22.0 AND education==9th THEN income=<=50K 
IF age<=22.0 AND education==10th THEN income=<=50K 
IF age<=22.0 AND education==Assoc-voc THEN income=<=50K 
IF age<=22.0 AND education==Masters THEN income=<=50K 
IF age<=22.0 AND marital-status==Divorced THEN income=<=50K 
IF age<=22.0 AND marital-status==Married-spouse-absent THEN income=<=50K 
IF age<=22.0 AND marital-status==Separated THEN income=<=50K 
IF age<=22.0 AND marital-status==Widowed THEN income=<=50

In [14]:
previsoes = Orange.evaluation.testing.TestOnTestData(base_treinamento, base_teste, [lambda testdata: regras_censo])

In [15]:
previsoes

In [16]:
Orange.evaluation.CA(previsoes)

array([0.82199017])

Utilizando Majority

In [17]:
majority = Orange.classification.MajorityLearner()
previsoes = Orange.evaluation.testing.TestOnTestData(base_censo, base_censo,[majority])
Orange.evaluation.CA(previsoes)

array([0.75919044])

In [18]:
from collections import Counter
Counter(str(registro.get_class()) for registro in base_censo)

Counter({'<=50K': 24720, '>50K': 7841})

In [19]:
24720 / (24720+7841)

0.7591904425539756